# Prepare network data for bicycle networks analysis
## Project: Bicycle network analysis with Gourab, Sayat, Tyler, Michael, Roberta

This notebook downloads bicycle and street network data sets and prepares them for analysis.

Contact: Michael Szell (michael.szell@gmail.com)  
Created: 2020-06-09  
Last modified: 2020-07-14

## Preliminaries

### Parameters

Make sure there is a cities.csv in the parameters folder, over which we will iterate.

In [ ]:
debug = False # If True, will produce plots and/or verbose output to double-check

osmnxparameters = {'car30':{'network_type':'drive', 'custom_filter':'["maxspeed"~"30"]'},
                   'carall':{'network_type':'drive', 'custom_filter':''}
                  }  
# Special case 'biketrack': "cycleway"~"track" OR "highway"~"cycleway" OR "bicycle"~"designated" OR "cycleway:right=track" OR "cycleway:left=track"
# Special case 'bikeable': biketrack OR car30
# See: https://wiki.openstreetmap.org/wiki/Key:cycleway#Cycle_tracks

### Setup

In [ ]:
%run -i path.py
%run -i setup.py

%matplotlib inline
%load_ext watermark
%watermark -n -v -m -g -iv

To check for nominatimstring:
* Go to e.g. https://nominatim.openstreetmap.org/search.php?q=paris%2C+france&polygon_geojson=1&viewbox= and enter the search string. If a correct polygon (or multipolygon) pops up it should be fine.

To get shapefiles:
* Go to [Overpass](overpass-turbo.eu), to the city, and run:
    `relation["boundary"="administrative"]["name:en"="Copenhagen Municipality"]({{bbox}});(._;>;);out skel;`
* Export: Download as GPX
* Use QGIS to create a polygon, with Vector > Join Multiple Lines, and Processing Toolbox > Polygonize (see https://gis.stackexchange.com/questions/98320/connecting-two-line-ends-in-qgis-without-resorting-to-other-software and https://gis.stackexchange.com/questions/207463/convert-a-line-to-polygon)

### Functions

In [ ]:
%run -i functions.py

## Download and wrangle data

### Networks

In [ ]:
for placeid, placeinfo in cities.items():
    placepath = PATH["data"] + placeid + "/"
    if placeinfo["nominatimstring"] != '':
        location = ox.gdf_from_place(placeinfo["nominatimstring"])
        location = shapely.geometry.shape(location['geometry'][0])
    else:
        # https://gis.stackexchange.com/questions/113799/how-to-read-a-shapefile-in-python
        shp = fiona.open(placepath + placeid + ".shp")
        first = shp.next()
        location = shapely.geometry.shape(first['geometry'])
    
    # Car stuff
    parameterid = 'carall'
    G_carall = ox.graph_from_polygon(location, 
                               network_type = osmnxparameters[parameterid]['network_type'],
                               custom_filter = (osmnxparameters[parameterid]['custom_filter']),
                               retain_all = True,
                               simplify = False)
    ox_to_csv(G_carall, placepath, placeid, parameterid)
    
    parameterid = 'car30'
    G_car30 = ox.graph_from_polygon(location, 
                               network_type = osmnxparameters[parameterid]['network_type'],
                               custom_filter = (osmnxparameters[parameterid]['custom_filter']),
                               retain_all = True,
                               simplify = False)
    ox_to_csv(G_car30, placepath, placeid, parameterid)
    
    # Special cases biketrack and bikeable
    G1 = ox.graph_from_polygon(location,
                               network_type = 'bike',
                               custom_filter = ('["cycleway"~"track"]'),
                               retain_all = True,
                               simplify = False)
    G2 = ox.graph_from_polygon(location, 
                               network_type = 'bike',
                               custom_filter = ('["highway"~"cycleway"]'),
                               retain_all = True,
                               simplify = False)
    G3 = ox.graph_from_polygon(location, 
                               network_type = 'bike',
                               custom_filter = ('["bicycle"~"designated"]'),
                               retain_all = True,
                               simplify = False)
    G4 = ox.graph_from_polygon(location, 
                               network_type = 'bike',
                               custom_filter = ('["cycleway:right"~"track"]'),
                               retain_all = True,
                               simplify = False)
    G5 = ox.graph_from_polygon(location, 
                               network_type = 'bike',
                               custom_filter = ('["cycleway:left"~"track"]'),
                               retain_all = True,
                               simplify = False)

    parameterid = 'biketrack'
    G_biketrack = nx.compose(G5, nx.compose(G1, nx.compose(G2, nx.compose(G3, G4))))
    ox_to_csv(G_biketrack, placepath, placeid, parameterid)
    
    parameterid = 'bikeable'
    G_bikeable = nx.compose(G_biketrack, G_car30)
    ox_to_csv(G_bikeable, placepath, placeid, parameterid)
    
    parameterid = 'biketrackcarall'
    G_biketrackcarall = nx.compose(G_biketrack, G_carall)
    ox_to_csv(G_biketrackcarall, placepath, placeid, parameterid)
    
    if debug:
        print(len(G_carall.vs), len(G_carall.es))
        print(len(G_biketrack.vs), len(G_biketrack.es))
        print(len(G_biketrackcarall.vs), len(G_biketrackcarall.es))
        print(len(G_biketrackcarall.vs)-len(G_biketrack.vs), len(G_biketrackcarall.es)-len(G_biketrack.es))